In [11]:
import h5py
import glob
from tqdm import tqdm
from mahjong.ReinforcementLearning.experience import ExperienceBuffer
from mahjong.models.model import DiscardModel, KongModel, PongModel

import torch
import torch.nn as nn

In [2]:
x, y, discard = ExperienceBuffer().read_experience('experiment_2021_04_06_14_21_07.h5')

In [3]:
x.shape

(2978, 195, 34, 1)

In [31]:
torch.tensor(y).dtype

torch.int64

In [18]:
torch.tensor(x).dtype

torch.float32

Data processing
1. Mulitply the discard by reward
2. Remove those record with y = 0

In [16]:
torch.tensor(discard).dtype

torch.int64

In [41]:
import h5py
import glob
from tqdm import tqdm
from mahjong.ReinforcementLearning.experience import ExperienceBuffer
from mahjong.models.model import DiscardModel, KongModel, PongModel

import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hyper-parameters
lr = 0.01
discount_factor=0.99
epsilon=0.5

def get_discounted_reward(r):
    discounted_r = (r - r.mean())/(r.std() + 1e-7)
    return discounted_r

def preprocess_data(s, a, r):
    s = torch.tensor(s, dtype=torch.float32, device=device)
    a = torch.tensor(a, dtype=torch.long, device=device)
    r = torch.tensor(r, dtype=torch.float32, device=device)
    discounted_r = get_discounted_reward(r)
    return s, a, discounted_r

# Training
exp_paths = glob.glob('./mahjong/data/*.h5')
exp_buffer = ExperienceBuffer()
model = DiscardModel(device).model
loss_fn = nn.CrossEntropyLoss(reduction='none')
optim = torch.optim.Adam(model.parameters(), lr=lr)

for exp in tqdm(exp_paths):
    states, rewards, actions = exp_buffer.read_experience(exp)
    states, actions, rewards = preprocess_data(states, actions, rewards)
    action_logits = model(s)
    
    optim.zero_grad()
    loss = rewards * loss_fn(action_logits, self.ep_a)
    loss = loss.mean()
    loss.backward()
    optim.step()

0it [00:00, ?it/s]

Loaded checkpoint discard-ep17-val_acc_0.7356-val_f1_0.7356.pth


1. Load model
2. Training using SGD with small lr